In [1]:
import datasets

In [1]:
import datasets
import pathlib
import itertools
import more_itertools
import rich
import torch
import transformers
import sys
print(pathlib.Path.cwd())
print(pathlib.Path.cwd().parent)
sys.path.append(str(pathlib.Path.cwd().parent))
import libs_data.data_commonsense_qa_few_shot
import libs_data.lib_commonsense_qa
import lib_trl_utils
import bin_main
import peft
import accelerate

FEW_SHOT = libs_data.data_commonsense_qa_few_shot.FEW_SHOT

/home/mila/g/gagnonju/Marg-Li-CoT/with_trl/scratchpad
/home/mila/g/gagnonju/Marg-Li-CoT/with_trl


In [3]:
split = "train"
dataset = datasets.load_dataset("commonsense_qa", split=split)
batch = [libs_data.lib_commonsense_qa.CommonSenseQAMC._prep_hf_ds(x) for x in itertools.islice(dataset, 1)]
batch

Found cached dataset commonsense_qa (/home/mila/g/gagnonju/.cache/huggingface/datasets/commonsense_qa/default/1.0.0/28d68f56649a7f0c23bc68eae850af914aa03f95f810011ae8cf58cc5ff5051b)


[{'question': 'Q: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?\nAnswer Choices:\n(A) ignore\n(B) enforce\n(C) authoritarian\n(D) yell at\n(E) avoid\nA:',
  'answer': 'A'}]

In [4]:
# MODEL_NAME = "EleutherAI/gpt-j-6B"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
DEVICE = 1
# model = transformers.AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B", torch_dtype=torch.bfloat16).to(1)
# tokenizer = transformers.AutoTokenizer.from_pretrained(model.config._name_or_path)
peft_config_dict = bin_main.DEFAULT_PEFT_CONFIG.copy()
peft_config_dict["task_type"] = peft.TaskType.CAUSAL_LM

model, forward_tokenizer, prediction_tokenizer = lib_trl_utils.init_model(
    model_name=MODEL_NAME,
    use_peft=True,
    peft_config_dict=peft_config_dict,
    peft_qlora_mode=False,
    precision=torch.bfloat16,
)
DEVICE = 0
accelerator = accelerate.Accelerator()
model = accelerator.prepare(model)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.


(0/1): trainable params: 8392705 || all params: 6746808321 || trainable%: 0.12439518955766107

In [4]:

gen_kwargs = bin_main.DEFAULT_GEN_KWARGS.copy()
gen_kwargs["num_return_sequences"] = 1
rich.print(gen_kwargs)
batch = [libs_data.lib_commonsense_qa.CommonSenseQAMC._prep_hf_ds(x) for x in itertools.islice(dataset, 1)]
keys = batch[0].keys()
unzipped_batch = {}

for k in keys:
    unzipped_batch[k] = [x[k] for x in batch]

preped_batch = libs_data.lib_commonsense_qa._tok_detok(
    batch=unzipped_batch,
    any_tokenizer=forward_tokenizer, 
    question_prefix=FEW_SHOT.strip() + "\n\n", 
    question_suffix=""
)

padded = prediction_tokenizer.pad(
    dict(input_ids=preped_batch["question_tok"]), 
    return_tensors="pt", 
    padding=True,
).to(DEVICE)

num_toks = padded["input_ids"].shape[1]
gen_kwargs["synced_gpus"] = False

print(padded["input_ids"].shape)
print("Query: `" + more_itertools.one(prediction_tokenizer.batch_decode(padded["input_ids"])) + "`")

{
    'repetition_penalty': 1,
    'min_new_tokens': 5,
    'top_k': 0.0,
    'top_p': 1.0,
    'early_stopping': True,
    'synced_gpus': True,
    'do_sample': True,
    'temperature': 1.0,
    'num_return_sequences': 1,
    'max_new_tokens': 200
}

NameError: name 'forward_tokenizer' is not defined

In [12]:
with model.pretrained_model.disable_adapter():
    gen_kwargs["repetition_penalty"] = 1
    generated = accelerator.unwrap_model(model).generate(**padded, **gen_kwargs)
    num_toks = padded["input_ids"].shape[1]
    print(prediction_tokenizer.batch_decode(generated[:, :]))
    model = model.eval()
    print("\n>>> " + "\n>>> ".join([x.split("\n", 1)[0] for x in prediction_tokenizer.batch_decode(generated[:, num_toks:])]))


["<s> Q: Why do I eat just enough when eating breakfast?\nAnswer Choices:\n(A) gas\n(B) full stomach\n(C) feeling satisfied (CORRECT)\n(D) have energy\n(e) bloated\nA: The answer must be a reason for eating just enough when eating breakfast. Eating just enough is a way to feel satisfied. Therefore, the answer is feeling satisfied (C).\n\nQ: How is a dog likely to communicate with another dog?\nAnswer Choices:\n(A) reproduce\n(B) bark (CORRECT)\n(C) do many things\n(D) clone\n(E) jump up\nA: The answer must be a way for a dog to communicate with another dog. Dogs bark to communicate with each other. Therefore, the answer is bark (B).\n\nQ: If a person is trying to weasel out of punishment for a crime, where are they likely to be?\nAnswer Choices:\n(A) court room (CORRECT)\n(B) cherry tree\n(C) chicken coop\n(D) natural history museum\n(E) jail\nA: The answer must be a place where a person would be trying to get out of punishment for a crime. The answer is court room (A).\n\nQ: What woul